In [27]:
from method import *
from public import *
from WindPy import w
import math


In [28]:

# 处理data
unit_value = Method.mergeSameSheet(file_name="固收净值", sheet_name="单位净值", path="Input_Data", header=[0,1], index_col=[0])
accu_value = Method.mergeSameSheet(file_name="固收净值", sheet_name="累计净值", path="Input_Data", header=[0,1], index_col=[0])

history_unit_value = pd.read_excel(Public.readFile("单位净值历史数据.xlsx"))
history_accu_value = pd.read_excel(Public.readFile("单位净值历史数据.xlsx"))
code_file = pd.read_excel(Public.readFile("统一三处代码.xlsx"))

code_SZ = code_file[["金富1号"]].T # 金富1号代码
unit_value.columns=code_SZ.values[0]
accu_value.columns=code_SZ.values[0]


start_date = unit_value.index[0]
end_date = unit_value.index[-1]
try:
    w.start("username=W1457909349;password=glh346glh;sitename=NJDX", waitTime=120)
    index_data = w.wsd("000906.SH", "close", start_date, end_date)
    w.close()
except:
    print("Wind Connect Error")


date_pd = []
for i in index_data.Times:
    date_pd.append(pd.Timestamp(i))
index_value = pd.DataFrame(index_data.Data[0], columns=["中证800"], index=date_pd)

Current Files: ['20221216-固收净值.xlsx', '20221215-固收净值.xlsx']
Current Files: ['20221216-固收净值.xlsx', '20221215-固收净值.xlsx']
Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2021 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [29]:

unit_value=pd.concat([unit_value, index_value], ignore_index = False)
unit_value = unit_value.groupby(unit_value.index).first() 

accu_value=pd.concat([accu_value, index_value], ignore_index = False)
accu_value = accu_value.groupby(accu_value.index).first() 


In [30]:

unit_value.insert(0, "日期", unit_value.index)
# unit_value.insert(0, "Unnamed: 0", unit_value.index)
accu_value.insert(0, "日期", accu_value.index)
# accu_value.insert(0, "Unnamed: 0", accu_value.index)
unit_value = unit_value.fillna(0)
accu_value = accu_value.fillna(0)
unit_value = unit_value[unit_value["中证800"] != 0]
accu_value = accu_value[accu_value["中证800"] != 0]

history_unit_value = pd.concat([history_unit_value, unit_value], ignore_index = True)
history_accu_value = pd.concat([history_accu_value, accu_value], ignore_index = True)
history_unit_value = history_unit_value.fillna(0)
history_accu_value = history_accu_value.fillna(0)


history_accu_value=Method.fillWithLastDay(history_accu_value, start_col=2, start_row=2)
history_unit_value=Method.fillWithLastDay(history_unit_value, start_col=2, start_row=2)
# return history_unit_value, history_accu_value
history_accu_value.to_csv("Output_Data/mid_1.csv")
history_unit_value.to_csv("Output_Data/mid_2.csv")


Start Position Col:  2 Row:  2 DataFrame Shape: (748, 156)
Start Position Col:  2 Row:  2 DataFrame Shape: (748, 156)


In [135]:
df_单位净值=pd.read_csv(Public.readFile("mid_1.csv"), index_col=0)
lst_最新一天单位净值 = []
lst_最新一天的日期 = []
for j in range(2,df_单位净值.shape[1]):
    idx = np.where(df_单位净值.iloc[:,j] !=0.0)[0][-1]
    lst_最新一天单位净值 = lst_最新一天单位净值 + [df_单位净值.iloc[idx,j]]
    lst_最新一天的日期 = lst_最新一天的日期 + [df_单位净值.index[idx]]
    
df_最新一天单位净值 = pd.DataFrame(df_单位净值.iloc[-1,2:].T)  #先粗略地用最后一行的代表最新的单位净值
df_最新一天单位净值["产品代码"] = df_最新一天单位净值.index
df_最新一天单位净值.columns = ["最新一天单位净值","产品代码"]
df_最新一天单位净值["最新一天单位净值"] = lst_最新一天单位净值  #再替换成真正的最新一天的单位净值
df_最新一天单位净值["最新一天的日期"] = lst_最新一天的日期
df_最新一天单位净值["最新一天的日期"] = [str(i).split(" ")[0] for i in df_最新一天单位净值["最新一天的日期"]]



#开始填充底层资产sample
df_底层资产sample = pd.read_excel(Public.readFile("持仓日报表模板.xlsx"),sheet_name="底层资产模板")
df_底层资产sample.index = df_底层资产sample["产品代码"]
for i in range(0,df_底层资产sample.shape[0]):
    产品代码 = df_底层资产sample.iloc[i,1]
    df_底层资产sample.iloc[i,10] = float(df_最新一天单位净值.loc[df_最新一天单位净值["产品代码"] == 产品代码]["最新一天单位净值"]) 
#double
df_底层资产sample["成本价（元/份）"] = df_底层资产sample["投资金额（万元）"]/df_底层资产sample["持有份额（份）"]*10000
df_底层资产sample["持仓市值（万元）"] = df_底层资产sample["持有份额（份）"] * df_底层资产sample["单位净值（元/份）"]/10000

持仓总市值 = df_底层资产sample["持仓市值（万元）"].sum()
df_底层资产sample["持仓市值占总持仓比例"] = df_底层资产sample["持仓市值（万元）"] /持仓总市值
f1 = lambda x :'%.2f%%'  %  (x*100)
df_底层资产sample[["持仓市值占总持仓比例"]] = df_底层资产sample[["持仓市值占总持仓比例"]].applymap(f1)
df_底层资产sample["2022年浮动盈亏（万元）"]= (df_底层资产sample["单位净值（元/份）"] - df_底层资产sample["2021年12月31日单位净值"])*df_底层资产sample["持有份额（份）"]/10000

df_底层资产sample["已实现盈亏（万元）"].fillna(0,inplace=True)
df_底层资产sample["2022年盈亏比例"] = (df_底层资产sample["2022年浮动盈亏（万元）"]+df_底层资产sample["已实现盈亏（万元）"])/df_底层资产sample["投资金额（万元）"]
df_底层资产sample[["2022年盈亏比例"]] = df_底层资产sample[["2022年盈亏比例"]].applymap(f1)
df_底层资产sample["累计浮动盈亏（万元）"] = (df_底层资产sample["单位净值（元/份）"] - df_底层资产sample["成本价（元/份）"])*df_底层资产sample["持有份额（份）"]/10000

df_底层资产sample["累计盈亏比例"] = (df_底层资产sample["累计浮动盈亏（万元）"]+df_底层资产sample["已实现盈亏（万元）"])/df_底层资产sample["投资金额（万元）"]
df_底层资产sample[["累计盈亏比例"]] = df_底层资产sample[["累计盈亏比例"]].applymap(f1)


series_各管理人持仓 = df_底层资产sample.groupby(by=["产品管理人"])["投资金额（万元）"].sum()
df_各管理人持仓 = series_各管理人持仓.reset_index()
for i in range(0,df_底层资产sample.shape[0]):
    产品管理人 = df_底层资产sample.iloc[i,3]
    df_底层资产sample.iat[i,-5]= df_各管理人持仓.loc[df_各管理人持仓["产品管理人"] == 产品管理人]["投资金额（万元）"]

for i in range(0,df_底层资产sample.shape[0]):
    if math.isnan(df_底层资产sample["产品规模（总份额）"].iloc[i]):
        df_底层资产sample.iloc[i,-3]=df_底层资产sample["产品规模（总份额）"].iloc[i]
    else:
        df_底层资产sample.iloc[i,-3] = format((df_底层资产sample["持有份额（份）"].iloc[i] / df_底层资产sample["产品规模（总份额）"].iloc[i]),".2%")
        
#填充号的底层资产sample就是底层资产私募配置情况表了
df_底层资产私募配置情况 = df_底层资产sample
df_hh = df_底层资产私募配置情况.fillna(0,inplace=False)
idx = []
for i in range(0,64):
    if df_hh.iloc[i,0] != 0:
        idx = idx + [i]
    else:
        df_hh.iloc[i,0] = df_hh.iloc[i,0]
        
for i in range(0,idx[1]):
    df_hh.iloc[i,0] = "一对一产品"
for i in range(idx[1],df_hh.shape[0]):
    df_hh.iloc[i,0] = "一对多产品"

df_底层资产私募配置情况.iloc[:,0] = df_hh.iloc[:,0]
# df_底层资产私募配置情况
#df_底层资产私募配置情况.head(2)

In [136]:
history_accu_value=pd.read_csv("Output_Data/mid_1.csv", index_col=0)
history_unit_value=pd.read_csv("Output_Data/mid_2.csv", index_col=0)


In [137]:
股票多头_index = np.where((history_accu_value.iloc[[1]] == "股票多头") == True)[1]
指数增强_index = np.where((history_accu_value.iloc[[1]] == "指数增强") == True)[1]
空气指增_index = np.where((history_accu_value.iloc[[1]] == "空气指增") == True)[1]
量化择时_index = np.where((history_accu_value.iloc[[1]] == "量化择时") == True)[1]
量化对冲_index = np.where((history_accu_value.iloc[[1]] == "量化对冲") == True)[1]
多策略灵活配置_index = np.where((history_accu_value.iloc[[1]] == "多策略灵活配置") == True)[1]
宏观对冲_index = np.where((history_accu_value.iloc[[1]] == "宏观对冲") == True)[1]
量化期货_index = np.where((history_accu_value.iloc[[1]] == "量化期货") == True)[1]

In [138]:
股票多头_unit_value = history_unit_value.iloc[:,股票多头_index]
股票多头_unit_value = history_unit_value[["中证800"]].join(股票多头_unit_value)
股票多头_unit_value = history_unit_value[["日期"]].join(股票多头_unit_value)


In [152]:
dataFrame = 股票多头_unit_value


result_df = pd.DataFrame

date = dataFrame.iloc[:,[0]]
start_date = date.iloc[2]
end_data = date.iloc[-1]
index = dataFrame.iloc[:,[1]]

for i in range(2, dataFrame.shape[1]):
    product = dataFrame.iloc[:,[i]]
    product_name = product.iloc[0]
    product_value = product.iloc[2:dataFrame.shape[0]]
    # 对每个产品计算
    accu_profit_rate = float(df_底层资产私募配置情况.loc[df_底层资产私募配置情况["产品代码"]==dataFrame.columns[i]]["累计盈亏比例"].values[0][:-1])
    
    print(product_value)
    



    SX1811
2        0
3        0
4        0
5        0
6        0
..     ...
743  1.604
744  1.604
745  1.604
746    1.6
747    1.6

[746 rows x 1 columns]
     SGW516
2         0
3         0
4         0
5         0
6         0
..      ...
743  1.9367
744  1.9367
745  1.9367
746  1.9367
747  1.9259

[746 rows x 1 columns]
    NB988A
2        0
3        0
4        0
5        0
6        0
..     ...
743  1.044
744  1.044
745  1.044
746  1.044
747  1.044

[746 rows x 1 columns]
     QA116B
2         0
3         0
4         0
5         0
6         0
..      ...
743  0.9658
744  0.9658
745  0.9658
746  0.9658
747  0.9573

[746 rows x 1 columns]
     QQ949A
2         0
3         0
4         0
5         0
6         0
..      ...
743  0.9245
744  0.9245
745  0.9245
746  0.9245
747  0.9208

[746 rows x 1 columns]
    S21804
2        0
3        0
4        0
5        0
6        0
..     ...
743  5.408
744  5.408
745  5.408
746   5.37
747   5.37

[746 rows x 1 columns]
    S69226
2        0
3     

In [101]:
def indexCal_1(dataFrame, *compare_dataFrame, **kwags) -> pd.DataFrame:
    """"""
    result_df = pd.DataFrame

    date = dataFrame.iloc[:,[0]]
    index = dataFrame.iloc[:,[1]]
    for i in range(2, dataFrame.shape[1]):
        product = dataFrame.iloc[:,[i]]


    return result_df

indexCal_1(股票多头_unit_value)

pandas.core.frame.DataFrame